https://www.youtube.com/watch?v=DbESHcCoWbM&t=13s

In [1]:
import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader, Dataset
import pytorch_lightning as pl
from PIL import Image
import os

In [2]:
# import sphere data
# define path
path = "/Users/maryclaremartin/Documents/Github/Research-Fall-2022/cali_camera_data"
dir_list = os.listdir(path)

# empty array for image names
image_names = []

# add image names from directory to a list
for image in dir_list:
    image_names.append(image)

# sort image titles
image_names = sorted(image_names)

In [3]:
#create array for images, segmentation images
images = []
seg_images = []

for image in image_names:
    if "img" in image:
        images.append(Image.open(os.path.join(path,image)))
    elif "seg" in image:
        seg_images.append(Image.open(os.path.join(path,image)))

In [4]:
# define dataset
class sphereDataset(Dataset):
    def __init__(self, seg_images, images, transform=transforms.ToTensor(), target_transform=transforms.ToTensor()):
        self.seg_images = seg_images
        self.images = images
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.seg_images)

    def __getitem__(self, idx):
        image = images[idx]
        seg = seg_images[idx]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            seg = self.target_transform(seg)
        return image, seg
    

In [7]:
class ImageClassifier(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(3,1,1)
        #self.l1 = nn.Linear(800 * 600, 64)
        #self.l2 = nn.Linear(64, 64)
        #self.l3 = nn.Linear(800 * 600,10)
        
        self.loss = nn.CrossEntropyLoss()
    
    def forward(self, x):
        #h1 = nn.functional.relu(self.l1(x))
        #h2 = nn.functional.relu(self.l2(h1))
        #logits = self.l3(h2)
        logits = nn.functional.relu(self.conv(x))
        return logits
    
    def configure_optimizers(self):
        optimizer = optim.SGD(self.parameters(), lr=1e-2) 
        return optimizer
    
    def training_step(self, batch, batch_idx):
        x, y = batch

        # flatten
        #b = x.size(0)
        #print('b:', b)
        #x = x.view(b, -1)
        
        #c = y.size(0)
        #y = y.view(c, -1)
        
        # forward
        logits = self(x)
        
        # compute the objective function
        J = self.loss(logits, y)
        
        return J
        
    def train_dataloader(self):
        train_data = sphereDataset(seg_images, images)
        train_loader = DataLoader(train_data, batch_size=1)
        return train_loader
    
model = ImageClassifier()
        

In [8]:
trainer = pl.Trainer(max_epochs = 10)
trainer.fit(model)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type             | Params
------------------------------------------
0 | conv | Conv2d           | 4     
1 | loss | CrossEntropyLoss | 0     
------------------------------------------
4         Trainable params
0         Non-trainable params
4         Total params
0.000     Total estimated model params size (MB)
/Users/maryclaremartin/opt/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:225: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

[W NNPACK.cpp:51] Could not initialize NNPACK! Reason: Unsupported hardware.
`Trainer.fit` stopped: `max_epochs=10` reached.


random stuff for figuring out sizes

In [9]:
#define dataset
sphereDataset = sphereDataset(seg_images, images)

In [10]:
sphereDataLoader = DataLoader(sphereDataset, batch_size = 5, shuffle = False)

In [11]:
input = next(iter(sphereDataLoader))

In [12]:
input1 = input[0]

In [13]:
input2 = input[1]

In [16]:
m = nn.Conv2d(3,1,3)

In [17]:
output = m(input1)

In [18]:
output.size()

torch.Size([5, 1, 598, 798])